In [8]:
# 필요한 라이브러리 설치
# !pip install transformers pandas torch gradio requests beautifulsoup4

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from bs4 import BeautifulSoup
import requests
import gradio as gr
import re

In [9]:
# 1. 기사 크롤링 함수
def extract_article_content(url):
    """
    주어진 URL에서 기사 본문을 추출.
    """
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.text, 'html.parser')

        # 뉴스 사이트별 본문 CSS 셀렉터
        selectors = [
            "div#dic_area",  # 네이버 뉴스
            "div.article_body",  # 특정 뉴스 사이트
            "div#articleBodyContents",  # 다음 뉴스
            "article",  # 일반적인 <article> 태그
            "div.content",  # 기타 사이트
        ]

        # 본문 추출
        for selector in selectors:
            content = soup.select_one(selector)
            if content and content.get_text(strip=True):
                return content.get_text(separator=" ", strip=True)
        
        return "기사를 읽을 수 없습니다. 링크를 확인하세요."
    except Exception as e:
        return f"오류 발생: {e}"

In [10]:
# 2. 전처리 함수
def preprocess_text(text):
    """
    텍스트를 전처리하여 모델 입력으로 적합하게 변환.
    """
    text = re.sub(r'\s+', ' ', text)  # 연속된 공백 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    return text.strip()[:512]  # 512 토큰 제한

In [11]:
# 3. 모델 초기화 함수
def initialize_models():
    """
    분류기와 요약 모델 초기화.
    """
    # Hugging Face 한국어 분류 모델 (NSMC 데이터 기반)
    model_name = "beomi/kcbert-base"  # 예: 감정 분류 모델
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

    # 한국어 요약 모델
    summarizer = pipeline("summarization", model="paust/pko-t5-base", tokenizer="paust/pko-t5-base")
    
    return classifier, summarizer

In [12]:
# 4. 기사 분석 함수
def analyze_article(url):
    """
    URL에서 기사를 크롤링하고 요약 및 분류.
    """
    # 기사 본문 추출
    article_content = extract_article_content(url)
    if "오류 발생" in article_content or "기사를 읽을 수 없습니다" in article_content:
        return article_content, "분류 불가", "요약 불가"
    
    # 전처리
    article_content_clean = preprocess_text(article_content)

    # 모델 초기화
    classifier, summarizer = initialize_models()

    # 가짜 뉴스 판별
    try:
        classification_result = classifier(article_content_clean)[0]["label"]
        classification = "진짜 뉴스" if classification_result == "LABEL_0" else "가짜 뉴스"
    except Exception as e:
        classification = f"분류 실패: {e}"

    # 기사 요약
    try:
        summary_result = summarizer(article_content_clean, max_length=50, min_length=10, do_sample=False)
        summary = summary_result[0]["summary_text"]
    except Exception as e:
        summary = f"요약 실패: {e}"

    return article_content, classification, summary

In [13]:
# 5. Gradio 인터페이스 정의
def gradio_interface(url):
    article_content, classification, summary = analyze_article(url)
    return article_content, classification, summary

# Gradio 설정
interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="기사 링크", placeholder="뉴스 링크를 입력하세요"),
    outputs=[
        gr.Textbox(label="기사 본문"),
        gr.Textbox(label="가짜 뉴스 여부"),
        gr.Textbox(label="요약된 내용")
    ],
    title="가짜 뉴스 탐지 및 요약기",
    description="기사 링크를 입력하면 본문을 크롤링하고, 요약 및 가짜 뉴스 여부를 판단합니다."
)


In [14]:
# 실행
if __name__ == "__main__":
    interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://5c4bf886c5dfd23ac4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


c:\Python310\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\human-24\.cache\huggingface\hub\models--beomi--kcbert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initial